In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
raw_data = pd.read_csv('TV.csv')
raw_data.head()

,Unnamed: 0,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,1,s2,TV Show,Blood & Water,Unknown,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
1,4,s5,TV Show,Kota Factory,Unknown,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
2,8,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
3,15,s16,TV Show,Dear White People,Unknown,"Logan Browning, Brandon P. Bell, DeRon Horton,...",United States,"September 22, 2021",2021,TV-MA,4 Seasons,"TV Comedies, TV Dramas",Students of color navigate the daily slights a...
4,17,s18,TV Show,Falsa identidad,Unknown,"Luis Ernesto Franco, Camila Sodi, Sergio Goyri...",Mexico,"September 22, 2021",2020,TV-MA,2 Seasons,"Crime TV Shows, Spanish-Language TV Shows, TV ...",Strangers Diego and Isabel flee their home in ...


In [53]:
print(f'Dữ liệu Netflix có {raw_data.shape[1]} cột và {raw_data.shape[0]} dòng.')

Dữ liệu Netflix có 13 cột và 2013 dòng.


In [54]:
raw_data['date_added']=pd.to_datetime(raw_data['date_added'])
raw_data['year_added']=raw_data['date_added'].apply(lambda x: int(x.year))
raw_data['month_added']=raw_data['date_added'].apply(lambda x: int(x.month))

In [55]:
cast = pd.DataFrame(raw_data['cast'].apply(lambda x: str(x).split(', ')).tolist(),index=raw_data['show_id'])
cast=cast.stack().reset_index().drop('level_1',axis=1)
cast.rename(columns={0:'actor'},inplace=True)

In [56]:
director = pd.DataFrame(raw_data['director'].apply(lambda x: str(x).split(', ')).tolist(),index=raw_data['show_id'])
director=director.stack().reset_index().drop('level_1',axis=1)
director.rename(columns={0:'director'},inplace=True)

In [57]:
country = pd.DataFrame(raw_data['country'].apply(lambda x: str(x).split(', ')).tolist(),index=raw_data['show_id'])
country=country.stack().reset_index().drop('level_1',axis=1)
country.rename(columns={0:'country'},inplace=True)

In [58]:
genre = pd.DataFrame(raw_data['listed_in'].apply(lambda x: str(x).split(', ')).tolist(),index=raw_data['show_id'])
genre=genre.stack().reset_index().drop('level_1',axis=1)
genre.rename(columns={0:'genre'},inplace=True)

In [59]:
temp = cast.merge(director, on=['show_id'], how='inner')
temp1 = temp.merge(country, on=['show_id'], how='inner')
temp2 = temp1.merge(genre, on=['show_id'], how='inner')

In [60]:
data = temp2.merge(raw_data[['show_id','type','title','date_added','release_year','rating','duration',
                             'description','year_added','month_added']],
                             on='show_id',how='inner').iloc[:,[0,5,6,1,2,3,4,7,8,9,10,11,12,13]]

In [61]:
movies = data[data['type']=='Movie']
tv_shows = data[data['type']=='TV Show']
tv=tv_shows['duration'].apply(lambda x:int(str(x).split(' ')[0]))
tv_shows=tv_shows.drop('duration',axis=1)
tv_shows['seasons']=tv
tv_shows.head()

,show_id,type,title,actor,director,country,genre,date_added,release_year,rating,description,year_added,month_added,seasons
0,s2,TV Show,Blood & Water,Ama Qamata,Unknown,South Africa,International TV Shows,2021-09-24,2021,TV-MA,"After crossing paths at a party, a Cape Town t...",2021,9,2
1,s2,TV Show,Blood & Water,Ama Qamata,Unknown,South Africa,TV Dramas,2021-09-24,2021,TV-MA,"After crossing paths at a party, a Cape Town t...",2021,9,2
2,s2,TV Show,Blood & Water,Ama Qamata,Unknown,South Africa,TV Mysteries,2021-09-24,2021,TV-MA,"After crossing paths at a party, a Cape Town t...",2021,9,2
3,s2,TV Show,Blood & Water,Khosi Ngema,Unknown,South Africa,International TV Shows,2021-09-24,2021,TV-MA,"After crossing paths at a party, a Cape Town t...",2021,9,2
4,s2,TV Show,Blood & Water,Khosi Ngema,Unknown,South Africa,TV Dramas,2021-09-24,2021,TV-MA,"After crossing paths at a party, a Cape Town t...",2021,9,2


In [62]:
# movie=movies['duration'].apply(lambda x:int(str(x).split(' ')[0]))
# movies=movies.drop('duration',axis=1)
# movies['duration']=movie
# movies.head()

In [63]:
t = movies[(movies['type']!='Unknown')&(movies['director']!='Unknown')].groupby(['type','director']).agg(
    {'show_id':'nunique'}).reset_index().sort_values(
    'show_id',ascending=False).head(10)
t1=movies.merge(t,on=['type','director'],how='inner').groupby(['type','director']).agg(
    {'show_id_x':'nunique'}).reset_index().sort_values('show_id_x',ascending=False).head(15).rename(
    {'show_id_x':'Total Movies'},axis=1)
t1

,type,director,Total Movies


In [64]:
pd.options.display.max_colwidth = 200
title_and_cast_df = raw_data[['director','listed_in']]
title_and_cast_df = title_and_cast_df.dropna()
cast_names_counter = {}
for i, row in title_and_cast_df.iterrows():
    movie_cast = row['director']
    movie_cast_split = movie_cast.split(',')
    movie_cast_stripped = [name.strip() for name in movie_cast_split]
    for name in movie_cast_stripped:
        cast_names_counter[name] = cast_names_counter.get(name,0) + 1
dict(sorted(cast_names_counter.items(),key=lambda item:item[1], reverse = True))

{'Unknown': 1866,
 'Alastair Fothergill': 3,
 'Gautham Vasudev Menon': 2,
 'Rob Seidenglanz': 2,
 'Jung-ah Im': 2,
 'Shin Won-ho': 2,
 'Stan Lathan': 2,
 'Iginio Straffi': 2,
 'Andy Devonshire': 1,
 'Kemi Adetiba': 1,
 'Juan Carlos Medina': 1,
 'Inma Torrente': 1,
 'Bejoy Nambiar': 1,
 'Priyadarshan': 1,
 'Karthik Narain': 1,
 'Vasanth Sai': 1,
 'Karthik Subbaraj': 1,
 'Arvind Swamy': 1,
 'Rathindran R Prasad': 1,
 'Sarjun': 1,
 'Hsu Fu-chun': 1,
 'Glen Winter': 1,
 'Jan Holoubek': 1,
 'Simon Frederick': 1,
 'Michael Simon': 1,
 'Joe Berlinger': 1,
 'Bruce Sinofsky': 1,
 'Jay Oliva': 1,
 'Ally Pankiw': 1,
 'Takuya Igarashi': 1,
 'Daniel Minahan': 1,
 'Kongkiat Khomsiri': 1,
 'Glenn Weiss': 1,
 'Pass Patthanakumjon': 1,
 'Ange Basterga': 1,
 'Nicolas Lopez': 1,
 'Tharun Bhascker Dhaassyam': 1,
 'B. V. Nandini Reddy': 1,
 'Nag Ashwin': 1,
 'Sankalp Reddy': 1,
 'Richard Arellano': 1,
 'Juliet May': 1,
 'Maite Ruiz De Austri': 1,
 'Tony Collingwood': 1,
 'Sudha Kongara': 1,
 'Vignesh Shiva